# Data Processing

In [65]:
#Uncomment all the cells with !pip to install the dependencies
os.chdir("..")

In [66]:
from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse

import cv2
import imutils

In [42]:
#! pip install --upgrade imutils


# EDGE Detection, Finding Contours and Saving the processed file

In [43]:
# Please place your raw images under Raw_images folder(for testting there is already an image placed)
# Processed images can be found under processed folder
#For speeding up the edge detection, image is resized

In [44]:
from os import listdir
from os.path import isfile, join
raw_img='Raw_images'
onlyfiles = [ f for f in listdir(raw_img) if isfile(join(raw_img,f)) ]
print(onlyfiles)
processed_img='processed'


['Walmart.10.jpg']


In [45]:
for imglist in onlyfiles:
    print(raw_img+imglist)
    image = cv2.imread(raw_img+'/'+imglist)
    print(image.shape)
    #For speeding up the edge detection, image is resized
    ratio = image.shape[0] / 500.0 # Ratio of original to new height
    orig = image.copy()
    image = imutils.resize(image, height = 500)
    print(image.shape)
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    print("Edge Detection")
    cv2.imshow("Image", image)
    cv2.imshow("Edged", edged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    # loop over the contours
    for c in cnts:
         # approximate the contour
         peri = cv2.arcLength(c, True)
         approx = cv2.approxPolyDP(c, 0.01 * peri, True)
 
         # if our approximated contour has four points, then we
         # can assume that we have found our receipt
         if len(approx) == 4:
                  screenCnt = approx
                  break
    print("STEP 2: Find contours of paper")
    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
    cv2.imshow("Outline", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(processed_img+'/'+'processed'+imglist,warped)
    print("STEP 3: Apply perspective transform")
    cv2.imshow("Original", imutils.resize(orig, height = 650))
    cv2.imshow("Scanned", imutils.resize(warped, height = 650))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    

Raw_imagesWalmart.10.jpg
(3264, 2448, 3)
(500, 375, 3)
Edge Detection
STEP 2: Find contours of paper
STEP 3: Apply perspective transform




# Finding Contours

threshold_local(image, block_size, method='gaussian', offset=0, mode='reflect', param=None, cval=0)[source]
Compute a threshold mask image based on local pixel neighborhood.

Also known as adaptive or dynamic thresholding. The threshold value is the weighted mean for the local neighborhood of a pixel subtracted by a constant. Alternatively the threshold can be determined dynamically by a given function, using the ‘generic’ method.

Parameters
image(N, M) ndarray
Input image.

block_sizeint
Odd size of pixel neighborhood which is used to calculate the threshold value (e.g. 3, 5, 7, …, 21, …).

method{‘generic’, ‘gaussian’, ‘mean’, ‘median’}, optional
Method used to determine adaptive threshold for local neighbourhood in weighted mean image.

‘generic’: use custom function (see param parameter)

‘gaussian’: apply gaussian filter (see param parameter for custom sigma value)

‘mean’: apply arithmetic mean filter

‘median’: apply median rank filter

By default the ‘gaussian’ method is used.

offsetfloat, optional
Constant subtracted from weighted mean of neighborhood to calculate the local threshold value. Default offset is 0.

mode{‘reflect’, ‘constant’, ‘nearest’, ‘mirror’, ‘wrap’}, optional
The mode parameter determines how the array borders are handled, where cval is the value when mode is equal to ‘constant’. Default is ‘reflect’.

param{int, function}, optional
Either specify sigma for ‘gaussian’ method or function object for ‘generic’ method. This functions takes the flat array of local neighbourhood as a single argument and returns the calculated threshold for the centre pixel.

cvalfloat, optional
Value to fill past edges of input if mode is ‘constant’.

Returns
threshold(N, M) ndarray
Threshold image. All pixels in the input image higher than the corresponding pixel in the threshold image are considered foreground.

# Cropping the images to get the Core area

In [64]:
# file present in the same directory
%run -i "photocrop.py"

main function
setPath function
C:\Users\INDRA\Documents\DSP-Project\processed
cropPhotos function
C:\Users\INDRA\Documents\DSP-Project\processed


# Finding Bounding Boxes

In [47]:
def decode_predictions(scores, geometry):
         # grab the number of rows and columns from the scores volume, then
         # initialize our set of bounding box rectangles and corresponding
         # confidence scores
         (numRows, numCols) = scores.shape[2:4]
         rects = []
         confidences = []
 
         # loop over the number of rows
         for y in range(0, numRows):
                  # extract the scores (probabilities), followed by the
                  # geometrical data used to derive potential bounding box
                  # coordinates that surround text
                  scoresData = scores[0, 0, y]
                  xData0 = geometry[0, 0, y]
                  xData1 = geometry[0, 1, y]
                  xData2 = geometry[0, 2, y]
                  xData3 = geometry[0, 3, y]
                  anglesData = geometry[0, 4, y]
 
                  # loop over the number of columns
                  for x in range(0, numCols):
                           # if our score does not have sufficient probability,
                           # ignore it
                           if scoresData[x] < 0.01:
                                    continue
 
                           # compute the offset factor as our resulting feature
                           # maps will be 4x smaller than the input image
                           (offsetX, offsetY) = (x * 4.0, y * 4.0)
 
                           # extract the rotation angle for the prediction and
                           # then compute the sin and cosine
                           angle = anglesData[x]
                           cos = np.cos(angle)
                           sin = np.sin(angle)
 
                           # use the geometry volume to derive the width and height
                           # of the bounding box
                           h = xData0[x] + xData2[x]
                           w = xData1[x] + xData3[x]
 
                           # compute both the starting and ending (x, y)-coordinates
                           # for the text prediction bounding box
                           endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
                           endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
                           startX = int(endX - w)
                           startY = int(endY - h)
 
                           # add the bounding box coordinates and probability score
                           # to our respective lists
                           rects.append((startX, startY, endX, endY))
                           confidences.append(scoresData[x])
 
         # return a tuple of the bounding boxes and associated confidences
         return (rects, confidences)

In [48]:
image = cv2.imread("processed/1_processedWalmart.10.jpg")
print(type(image))
orig = image.copy()
(origH, origW) = image.shape[:2]
print((origH, origW))

<class 'numpy.ndarray'>
(900, 843)


In [49]:
#set new width and height and save the old to new ratio's

In [50]:
(newW, newH) = (320, 320)
rW = origW / float(newW)
rH = origH / float(newH)

In [51]:
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]
print(image.shape)
cv2.imshow("Image",image)
cv2.waitKey(0)

(320, 320, 3)


-1

# East text detector

In [52]:
layerNames = [
  "feature_fusion/Conv_7/Sigmoid",
  "feature_fusion/concat_3"]

In [53]:
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

In [54]:
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
(123.68, 116.78, 103.94), swapRB=True, crop=False)
print(blob)
print("First Blob: {}".format(blob.shape))

[[[[ 58.32      59.32      58.32     ...  73.32      73.32
     74.32    ]
   [ 60.32      58.32      60.32     ...  73.32      73.32
     74.32    ]
   [ 59.32      61.32      59.32     ...  73.32      72.32
     73.32    ]
   ...
   [ 22.32      17.32      20.32     ...  80.32      77.32
     79.32    ]
   [ 24.32      19.32      21.32     ...  77.32      78.32
     79.32    ]
   [ 21.32      20.32      21.32     ...  78.32      79.32
     80.32    ]]

  [[ 65.22      66.22      65.22     ...  80.22      80.22
     81.22    ]
   [ 67.22      65.22      67.22     ...  80.22      80.22
     81.22    ]
   [ 66.22      68.22      66.22     ...  80.22      79.22
     80.22    ]
   ...
   [ 29.220001  24.220001  27.220001 ...  87.22      84.22
     86.22    ]
   [ 31.220001  26.220001  28.220001 ...  84.22      85.22
     86.22    ]
   [ 28.220001  27.220001  28.220001 ...  85.22      86.22
     87.22    ]]

  [[ 78.06      79.06      78.06     ...  93.06      93.06
     94.06    ]
   [ 80

In [55]:
keypoints=blob
print(keypoints)
img = image.copy()
print(len(keypoints))
#for x in range(0,len(keypoints)):
 # img2=cv2.circle(img, (np.int(keypoints[x].pt[0]),np.int(keypoints[x].pt[1])), radius=np.int(keypoints[x].size), color=(0,255,0), thickness=1000)
#cv2.imshow("Image",img2)
#cv2.waitKey(0)

[[[[ 58.32      59.32      58.32     ...  73.32      73.32
     74.32    ]
   [ 60.32      58.32      60.32     ...  73.32      73.32
     74.32    ]
   [ 59.32      61.32      59.32     ...  73.32      72.32
     73.32    ]
   ...
   [ 22.32      17.32      20.32     ...  80.32      77.32
     79.32    ]
   [ 24.32      19.32      21.32     ...  77.32      78.32
     79.32    ]
   [ 21.32      20.32      21.32     ...  78.32      79.32
     80.32    ]]

  [[ 65.22      66.22      65.22     ...  80.22      80.22
     81.22    ]
   [ 67.22      65.22      67.22     ...  80.22      80.22
     81.22    ]
   [ 66.22      68.22      66.22     ...  80.22      79.22
     80.22    ]
   ...
   [ 29.220001  24.220001  27.220001 ...  87.22      84.22
     86.22    ]
   [ 31.220001  26.220001  28.220001 ...  84.22      85.22
     86.22    ]
   [ 28.220001  27.220001  28.220001 ...  85.22      86.22
     87.22    ]]

  [[ 78.06      79.06      78.06     ...  93.06      93.06
     94.06    ]
   [ 80

In [56]:
#start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
#end = time.time()

In [57]:
from imutils.object_detection import non_max_suppression

In [58]:
(rects, confidences) = decode_predictions(scores, geometry)
#print(confidences)
boxes = non_max_suppression(np.array(rects), probs=confidences)
print(boxes)

[[ 11  72  37  87]
 [103  79 146  93]
 [110   9 178  22]
 [121 128 193 142]
 [111 142 203 158]
 [ 52 249  96 262]
 [112 230 199 245]
 [ 53  16  79  32]
 [161  88 187 102]
 [108  64 172  77]
 [ 56 162  86 175]
 [100  41 195  56]
 [105  23 199  38]
 [ 13 176  37 191]
 [129 104 158 118]
 [ 48 136  71 150]
 [104 161 146 173]
 [104 118 144 132]
 [ 14 253  42 264]
 [ 42 226  71 239]
 [113 217 192 232]
 [ 49 120  82 135]
 [126  91 156 105]
 [ 60  28  94  44]
 [ 13  18  37  31]
 [118 167 192 182]
 [ 14 201  37 214]
 [156  76 187  90]
 [139 244 189 257]
 [ 52 236  96 250]
 [ 49  55  69  72]
 [ 12 238  43 251]
 [ 14  56  43  70]
 [129 116 181 130]
 [253   1 272  15]
 [ 44  95  65 112]
 [ 43  81  61 100]
 [ 64 289  87 302]
 [103 284 152 298]
 [  6  31  33  47]
 [ 41 175  72 190]
 [255  15 269  33]
 [256  55 268  74]
 [232 278 248 302]
 [139 295 189 310]
 [ 13 189  36 202]
 [257 274 271 299]
 [157 282 194 295]
 [102 273 141 287]
 [254 102 268 127]
 [ 42 109  69 125]
 [105 259 149 273]
 [ 31   2  6

In [63]:
for (startX, startY, endX, endY) in boxes:
         # scale the bounding box coordinates based on the respective
         # ratios
         
         #dX = int((endX - startX))*0.8
         #dY = int((endY - startY))*0.8
         startX = int(startX * rW)
         startY = int(startY * rH)
         endX = int(endX * rW)
         endY = int(endY * rH)
 
         # draw the bounding box on the image
         
         #dX = int((endX - startX))*0.8
         #dY = int((endY - startY))*0.8
         
 
         # apply padding to each side of the bounding box, respectively
        # startX = max(0, startX - dX)
        # startY = max(0, startY - dY)
        # endX = min(origW, endX + (dX * 2))
        # endY = min(origH, endY + (dY * 2))
        # print(type(startX),startY)
         cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2) 
        # extract the actual padded ROI
         #roi = orig[startY:endY, startX:endX]
 
# show the output image
cv2.imshow("Text Detection", orig)
cv2.waitKey(0)

-1